In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import seaborn as sns



In [ ]:
df = pd.read_csv('../data/converted/EDE_B121069907_K600_oxygen_b.csv')
df.set_index('Unnamed: 0', inplace=True)


In [ ]:
expected_interval = pd.Timedelta(minutes=1)
df.index = pd.to_datetime(df.index)
complete_time_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=expected_interval)
missing_timestamps = complete_time_range[~complete_time_range.isin(df.index)]
missing_timestamps

In [ ]:
df.loc[df.index>="2021-04-09 09:10"]

In [ ]:
df1 = pd.read_csv("../data/merged_oxt101.csv", index_col=0).drop(columns={'_water.csv'})
df2 = pd.read_csv("../data/merged_oxt102.csv", index_col=0).drop(columns={'_water.csv'})
df3 = pd.read_csv("../data/merged_oxt103.csv", index_col=0).drop(columns={'_water.csv'})

In [ ]:
df1.loc[df1.index>="2021-04-09 09:40"]

In [ ]:
df1 = df1.rename(columns={'EDE_B121069914_K600_nitrate.csv':'nitrate','EDE_B121069917_K600_phosphate.csv':'phosphate','EDE_B121069913_K600_ammonium.csv':'ammonium','EDE_B121069907_K600_oxygen_b.csv':'oxygen_b','EDE_B121069901_K600_oxygen_a.csv':'oxygen_a'})


In [ ]:
df1['avg_oxygen'] = df1[['oxygen_b', 'oxygen_a']].mean(axis=1)
df1.describe()

In [ ]:
df1['oxygen_b'].loc[(df1.index>="2021-10-31 02:00" )& (df1.index<="2021-11-01 00:00") ]

In [ ]:
# Define the bins and labels for 'High', 'Medium', 'Low'
bins = [0, 0.5, 2, 3]  # Define your own boundaries here
labels = ['Low', 'Medium', 'High']

# Create a new column based on 'Values' column
df1['oxygen_catagory'] = pd.cut(df1['avg_oxygen'], bins=bins, labels=labels)

# Display the updated DataFrame
df1

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

model = RandomForestRegressor()


In [ ]:
X = df1[['nitrate', 'phosphate', 'ammonium']]
y = df1['oxygen_b']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



In [ ]:
model.fit(X_train, y_train)


In [ ]:
pred = model.predict(X_test)


In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
pred1 = pred
pred1.set_index(y_test.index, inplace=True)
pred1

In [ ]:
merged_df = pd.merge(y_test, pred1, left_index=True, right_index=True, how='outer').rename(columns={0:'oxygen_b_pred'})
merged_df = merged_df.sort_index()
merged_df

In [ ]:
merged_df.tail(1000).plot.line(figsize=(30, 20))

In [ ]:
from sklearn.metrics import mean_absolute_error
mae_per_column = mean_absolute_error(y_test, pred1)


In [ ]:
mae_per_column

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model2 = DecisionTreeRegressor()


In [ ]:
model2.fit(X_train,y_train)

In [ ]:
pred2 = model2.predict(X_test)
pred2 = pd.DataFrame(pred2)

In [ ]:
pred2.set_index(y_test.index, inplace=True)
pred2

In [ ]:
merged_df2 = pd.merge(y_test, pred2, left_index=True, right_index=True, how='outer').rename(columns={0:'oxygen_b_pred'})
merged_df2 = merged_df2.sort_index()
merged_df2

In [ ]:
merged_df.tail(1000).plot.line(figsize=(30, 20))

In [ ]:
mae_per_column = mean_absolute_error(y_test, pred2)
mae_per_column

In [ ]:
dfs = df1.drop(columns=['avg_oxygen','oxygen_catagory'])
dfs

In [ ]:
import statsmodels.api as statsmodels
import patsy

statsmodels.tsa.seasonal_decompose(dfs, model='additive', filt=None, period=1, two_sided=True, extrapolate_trend=0)